In [1]:
import scipy.io as scio
import pandas as pd
import os
import numpy as np
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.functional import interpolate
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torch.utils.data as Data
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
from torch import einsum
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold, LeaveOneGroupOut
import copy
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, accuracy_score
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
from tqdm import tqdm, trange
from torcheeg.datasets.constants.emotion_recognition import format_region_channel_list
import random
import mne


In [2]:
train_data=torch.from_numpy(np.load(f'data_training/eright_and_cwrong_train_data.npy'))
train_dec_label=torch.from_numpy(np.load(f'data_training/eright_and_cwrong_train_dec_label.npy'))
train_emo_label=torch.from_numpy(np.load(f'data_training/eright_and_cwrong_train_emo_label.npy'))
test_data=torch.from_numpy(np.load(f'data_training/eright_and_cwrong_test_data.npy'))
test_dec_label=torch.from_numpy(np.load(f'data_training/eright_and_cwrong_test_dec_label.npy'))
test_emo_label=torch.from_numpy(np.load(f'data_training/eright_and_cwrong_test_emo_label.npy'))
train_data.shape,train_dec_label.shape,train_emo_label.shape,test_data.shape,test_dec_label.shape,test_emo_label.shape

(torch.Size([1698, 1, 62, 875]),
 torch.Size([1698]),
 torch.Size([1698]),
 torch.Size([425, 1, 62, 875]),
 torch.Size([425]),
 torch.Size([425]))

In [3]:
eeg_data_now = mne.io.read_raw_eeglab('F:/preprocessed_01102201.set', preload=True)
# 要删除的通道名字列表
channels_to_remove = ['M1', 'M2','HEO','VEO','Trigger']

# 使用 pick_channels 删除指定的通道
eeg_data_now.pick_channels(ch_names=[ch for ch in eeg_data_now.ch_names if ch not in channels_to_remove])

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


<RawEEGLAB | preprocessed_01102201.set, 62 x 164825 (659.3 s), ~78.1 MiB, data loaded>

In [4]:
LOCATION_LIST = [
    [ '-', '-', '-', 'FP1', 'FPZ', 'FP2', '-', '-', '-'],
    [ '-', '-', '-', 'AF3', '-', 'AF4', '-', '-', '-'],
    [ 'F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8'],
    [ 'FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'FT8'],
    [ 'T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 'C6', 'T8'],
    [ 'TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6', 'TP8'],
    [ 'P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8'],
    [ '-', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', '-'],
    [ '-', '-', 'CB1', 'O1', 'OZ', 'O2', 'CB2', '-', '-' ]
]
HEMISPHERE_LIST = [
    ['F7', 'F5', 'F3', 'F1'], 
    ['F2', 'F4', 'F6', 'F8'],
    [ 'FC5', 'FC3', 'FC1'],
    ['FC2', 'FC4', 'FC6'], 
    ['FP1',  'AF3'], 
    ['FP2', 'AF4', ],
    ['FPZ', 'FZ', 'FCZ', 'CZ', 'CPZ', 'PZ', 'POZ', 'OZ'],
    ['C5', 'C3', 'C1'], ['C2', 'C4', 'C6'], ['CP5', 'CP3', 'CP1'],
    ['CP2', 'CP4', 'CP6'], 
    ['P7', 'P5', 'P3', 'P1'], 
    ['P2', 'P4', 'P6', 'P8'],
    ['PO7','PO5', 'PO3', 'O1', 'CB1'], 
    ['PO4', 'PO6', 'PO8', 'O2', 'CB2'], 
    ['FT7','T7', 'TP7'],
    ['FT8','T8', 'TP8' ]
]
CHANNELS_LIST = eeg_data_now.ch_names

GENERAL_REGION_LIST = format_region_channel_list(CHANNELS_LIST, LOCATION_LIST)     #按行分

HEMISPHERE_REGION_LIST = format_region_channel_list(CHANNELS_LIST, HEMISPHERE_LIST)            #按区域分


In [5]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.module import Module
from torch.nn.parameter import Parameter


class ChannelAttention(nn.Module):
    def __init__(self, in_channelsnel, ratio=2):
        super(ChannelAttention, self).__init__()
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Conv2d(in_channelsnel,
                             in_channelsnel // ratio,
                             1,
                             bias=False)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Conv2d(in_channelsnel // ratio,
                             in_channelsnel,
                             1,
                             bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_pool_out = self.avg_pool(x)
        max_pool_out = self.max_pool(x)
        avg_pool_out = self.fc2(self.relu1(self.fc1(avg_pool_out)))
        max_pool_out = self.fc2(self.relu1(self.fc1(max_pool_out)))
        out = max_pool_out + avg_pool_out
        return self.sigmoid(out)


class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        assert kernel_size in (3, 7), 'Kernel size must be 3 or 7.'
        padding = 3 if kernel_size == 7 else 1
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        max_pool_out, _ = torch.max(x, dim=1, keepdim=True)
        avg_pool_out = torch.mean(x, dim=1, keepdim=True)
        out = torch.cat([avg_pool_out, max_pool_out], dim=1)
        out = self.conv1(out)
        return self.sigmoid(out)


class CBAMBlock(nn.Module):
    def __init__(self, in_channelsnel, ratio=2, kernel_size=7):
        super(CBAMBlock, self).__init__()
        self.cha_att = ChannelAttention(in_channelsnel, ratio=ratio)
        self.spa_att = SpatialAttention(kernel_size=kernel_size)

    def forward(self, x):
        out = x * self.cha_att(x)
        out = out * self.spa_att(out)
        return out


class PowerLayer(nn.Module):
    def __init__(self, kernel_size, stride):
        super(PowerLayer, self).__init__()
        self.pooling = nn.AvgPool2d(kernel_size=(1, kernel_size),
                                    stride=(1, stride))

    def forward(self, x):
        return torch.log(self.pooling(x.pow(2)))


class Aggregator():
    def __init__(self, region_list):
        self.region_list = region_list

    def forward(self, x):
        output = []
        for region_index in range(len(self.region_list)):
            region_x = x[:, self.region_list[region_index], :]
            aggr_region_x = torch.mean(region_x, dim=1)
            output.append(aggr_region_x)
        return torch.stack(output, dim=1)


class GraphConvolution(Module):
    def __init__(self, in_channels, out_channels, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.weight = Parameter(torch.FloatTensor(in_channels, out_channels))

        if bias:
            self.bias = Parameter(
                torch.zeros((1, 1, out_channels), dtype=torch.float32))
        else:
            self.register_parameter('bias', None)
        nn.init.xavier_uniform_(self.weight, gain=1.414)

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, x, adj):
        output = torch.matmul(x, self.weight) - self.bias
        output = F.relu(torch.matmul(adj, output))
        return output


class MECM_EDNet(nn.Module):
    def __init__(self,
                 region_list,
                 in_channels: int = 1,
                 num_electrodes: int = 32,
                 chunk_size: int = 128,
                 sampling_rate: int = 128,
                 num_T: int = 64,
                 hid_channels: int = 32,
                 dropout: float = 0.5,
                 pool_kernel_size: int = 16,
                 pool_stride: int = 4,
                 num_classes_dec: int = 2,
                num_classes_emo:int = 4):
        super(MECM_EDNet, self).__init__()
        self.region_list = region_list
        self.inception_window = [0.5, 0.25, 0.125]

        self.num_classes_dec = num_classes_dec
        self.num_classes_emo= num_classes_emo
        self.in_channels = in_channels
        self.num_electrodes = num_electrodes
        self.chunk_size = chunk_size
        self.sampling_rate = sampling_rate
        self.num_T = num_T
        self.hid_channels = hid_channels
        self.dropout = dropout
        self.pool_kernel_size = pool_kernel_size
        self.pool_stride = pool_stride
        self.in_channels = in_channels
        self.num_electrodes = num_electrodes

        self.t_block1 = self.temporal_block(
            self.in_channels, self.num_T,
            (1, int(self.inception_window[0] * self.sampling_rate)),
            self.pool_kernel_size, self.pool_stride)
        self.t_block2 = self.temporal_block(
            self.in_channels, self.num_T,
            (1, int(self.inception_window[1] * self.sampling_rate)),
            self.pool_kernel_size, self.pool_stride)
        self.t_block3 = self.temporal_block(
            self.in_channels, self.num_T,
            (1, int(self.inception_window[2] * self.sampling_rate)),
            self.pool_kernel_size, self.pool_stride)

        self.bn_t1 = nn.BatchNorm2d(self.num_T)
        self.bn_t2 = nn.BatchNorm2d(self.num_T)

        self.cbam = CBAMBlock(num_electrodes)

        self.conv1x1 = nn.Sequential(
            nn.Conv2d(num_T, num_T, kernel_size=(1, 1), stride=(1, 1)),
            nn.LeakyReLU(), nn.AvgPool2d((1, 2)))

        self.avg_pool = nn.AvgPool2d((1, 2))

        feature_dim = self.feature_dim
        self.local_filter_weight = nn.Parameter(torch.FloatTensor(
            self.num_electrodes, feature_dim),
                                                requires_grad=True)
        self.local_filter_bias = nn.Parameter(torch.zeros(
            (1, self.num_electrodes, 1), dtype=torch.float32),
                                              requires_grad=True)

        self.aggregate = Aggregator(self.region_list)
        num_region = len(self.region_list)

        self.global_adj = nn.Parameter(torch.FloatTensor(
            num_region, num_region),
                                       requires_grad=True)

        self.bn_g1 = nn.BatchNorm1d(num_region)
        self.bn_g2 = nn.BatchNorm1d(num_region)

        self.gcn = GraphConvolution(feature_dim, hid_channels)

        self.fc_dec = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(int(num_region * hid_channels), num_classes_dec))
        
        self.fc_emo = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(int(num_region * hid_channels), num_classes_emo))

        nn.init.xavier_uniform_(self.local_filter_weight)
        nn.init.xavier_uniform_(self.global_adj)

    def temporal_block(self, in_channels, out_channels, kernel_size,
                       pool_kernel_size, pool_stride):
        return nn.Sequential(
            nn.Conv2d(in_channels,
                      out_channels,
                      kernel_size=kernel_size,
                      stride=(1, 1)),
            PowerLayer(kernel_size=pool_kernel_size, stride=pool_stride))

    def forward(self, x):
        r'''
        Args:
            x (torch.Tensor): EEG signal representation, the ideal input shape is :obj:`[n, 1, 32, 128]`. Here, :obj:`n` corresponds to the batch size, :obj:`32` corresponds to :obj:`num_electrodes`, and :obj:`chunk_size` corresponds to :obj:`chunk_size`.

        Returns:
            torch.Tensor[number of sample, number of classes]: the predicted probability that the samples belong to the classes.
        '''
        t1 = self.t_block1(x)
        t2 = self.t_block2(x)
        t3 = self.t_block3(x)
        x = torch.cat((t1, t2, t3), dim=-1)

        x = self.bn_t1(x)

        x = x.permute(0, 2, 1, 3)
        x = self.cbam(x)
        x = self.avg_pool(x)

        x = x.flatten(start_dim=2)
        x = self.local_filter(x)
        x = self.aggregate.forward(x)
        adj = self.get_adj(x)
        x = self.bn_g1(x)

        x = self.gcn(x, adj)
        x = self.bn_g2(x)
        x = x.view(x.shape[0], -1)
        x1 = self.fc_dec(x)
        x2= self.fc_emo(x)
        return x1,x2,x

    @property
    def feature_dim(self):
        mock_eeg = torch.randn(
            (1, self.in_channels, self.num_electrodes, self.chunk_size))

        t1 = self.t_block1(mock_eeg)
        t2 = self.t_block2(mock_eeg)
        t3 = self.t_block3(mock_eeg)
        mock_eeg = torch.cat((t1, t2, t3), dim=-1)

        mock_eeg = self.bn_t1(mock_eeg)
        mock_eeg = self.conv1x1(mock_eeg)
        mock_eeg = self.bn_t2(mock_eeg)
        mock_eeg = mock_eeg.permute(0, 2, 1, 3)
        mock_eeg = mock_eeg.flatten(start_dim=2)
        return mock_eeg.shape[-1]

    def local_filter(self, x):
        w = self.local_filter_weight.unsqueeze(0).repeat(x.shape[0], 1, 1)
        x = F.relu(torch.mul(x, w) - self.local_filter_bias)
        return x

    def get_adj(self, x, self_loop=True):
        adj = torch.bmm(x, x.permute(0, 2, 1))
        num_nodes = adj.shape[-1]
        adj = F.relu(adj * (self.global_adj + self.global_adj.transpose(1, 0)))
        if self_loop:
            adj = adj + torch.eye(num_nodes).to(x.device)
        rowsum = torch.sum(adj, dim=-1)
        mask = torch.zeros_like(rowsum)
        mask[rowsum == 0] = 1
        rowsum += mask
        d_inv_sqrt = torch.pow(rowsum, -0.5)
        d_mat_inv_sqrt = torch.diag_embed(d_inv_sqrt)
        adj = torch.bmm(torch.bmm(d_mat_inv_sqrt, adj), d_mat_inv_sqrt)
        return adj

In [6]:
import numpy as np
import torch
import torch.nn as nn
from torch import nn
import torch.nn.functional as F
import numpy as np
import tqdm
import time
import time
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import random
from torch.utils import data
import argparse
import os, shutil
# from ResNet_model import ResNet
parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
import argparse
torch.set_default_tensor_type(torch.FloatTensor)
device = torch.device('cuda')
import torch.nn.functional as F
#减数分裂代码
def meiosis(signal):
    num_pieces=int(len(signal)/2)
    rand_subs_stre = random.sample(range(0, num_pieces*2), num_pieces*2)
    split=random.randint(1,signal.shape[-1]-2)
    new_signal = []
    new_signal1 = []
    new_signal2 = []
    for i in range(num_pieces):
        si = rand_subs_stre[i]
        sj = rand_subs_stre[i+num_pieces]
#         print(signal.shape)
        xi = np.concatenate([signal[si,:,:,:split],signal[sj,:,:,split:]],axis=2)
#         print(xi.shape)
        xj = np.concatenate([signal[sj,:,:,:split],signal[si,:,:,split:]],axis=2)
        new_signal1.append(xi)
        new_signal2.append(xj)
    new_signal = new_signal1 + new_signal2 #合并list
    new_signal = np.array(new_signal)
    return new_signal,split

G:\python310\lib\site-packages\torch\__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ..\torch\csrc\tensor\python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


In [7]:
class SupervisedContrastiveLoss(nn.Module):
    def __init__(self, temperature=0.5):
        super(SupervisedContrastiveLoss, self).__init__()
        self.temperature = temperature

    def forward(self, features, labels):
        # L2 normalize the features
        labels=labels.reshape(-1,1)
        features = F.normalize(features, dim=-1, p=2)

        # Compute the cosine similarity between features
        similarity_matrix = torch.matmul(features, features.T)

        # Build a mask to separate positive and negative pairs
        mask = torch.eq(labels, labels.mT).float()
        # Calculate the log probability of positive pairs
        log_prob = similarity_matrix - (mask * 1e9)

        # Compute the mean loss for positive pairs
        loss = -torch.log(torch.exp(log_prob).sum(1) / torch.exp(log_prob).sum(1).sum())

#         if np.isnan(loss).any():
#             print(torch.exp(log_prob).sum(1),log_prob)
            
        return loss.mean()

In [8]:
from tqdm import tqdm
def MECM_EDNet_learning(alpha,beta,gamma):


    # for i in range(len(model_name_list)):
    #     model_name=model_name_list[i]
#######################基本参数########################################
    device = torch.device("cuda:0")
#     print(class_weights)
    criterion1 = nn.CrossEntropyLoss()
    criterion2 = nn.CrossEntropyLoss()
    criterion3 = SupervisedContrastiveLoss()

    learning_rate = 0.0001
    epoch_size = 13

    save = True

    model_name = 'MeCM_EDNet'
###########################################################################
    # 创建正样本和负样本的索引列表
    positive_indices = (train_dec_label == 0).nonzero().squeeze()
    negative_indices = (train_dec_label == 1).nonzero().squeeze()

    print(positive_indices.shape,negative_indices.shape)
    # 定义批次大小
    batch_size1 = 17
    batch_size2 = 15

    # 创建正样本和负样本的数据增强的训练数据集
    # positive_dataset = TensorDataset(train_data[positive_indices], train_emo_label[positive_indices])
    # negative_dataset = TensorDataset(train_data[negative_indices], train_emo_label[negative_indices])
    positive_data=train_data[positive_indices]
    negative_data=train_data[negative_indices]

    enhanced_positive_data,spilt=meiosis(positive_data)
    enhanced_negative_data,spilt=meiosis(negative_data)

    enhanced_positive_data=torch.tensor(enhanced_positive_data)
    enhanced_negative_data=torch.tensor(enhanced_negative_data)
    train_pos_data=torch.cat((positive_data,enhanced_positive_data),axis=0)
    train_neg_data=torch.cat((negative_data,enhanced_negative_data),axis=0)

    #把emo标签拼接完成
    length1=int(np.floor(len(positive_indices)/2)*2+len(positive_indices))
    length2=int(np.floor(len(negative_indices)/2)*2+len(negative_indices))
    pos_emo_label=torch.cat((train_emo_label[positive_indices],train_emo_label[positive_indices]),axis=0)[:length1]
    neg_emo_label=torch.cat((train_emo_label[negative_indices],train_emo_label[negative_indices]),axis=0)[:length2]

    positive_dataset = TensorDataset(train_pos_data,pos_emo_label)
    negative_dataset = TensorDataset(train_neg_data,neg_emo_label)

    # 创建两个训练数据加载器，一个用于正样本，一个用于负样本
    positive_dataloader = DataLoader(positive_dataset, batch_size=batch_size1, shuffle=True)
    negative_dataloader = DataLoader(negative_dataset, batch_size=batch_size2, shuffle=True)

    #测试数据加载
    test_set = TensorDataset(test_data, test_dec_label,test_emo_label)
    test_loader = Data.DataLoader(test_set, batch_size=32)





    acc_all = []
    f1_all = []
    recall_all = []
    precision_all = []
    for k in range(5):

        model = MECM_EDNet(region_list=HEMISPHERE_REGION_LIST, chunk_size=875, num_T=250, num_electrodes=62, hid_channels=32, num_classes_dec=2,num_classes_emo=4).to(device)


        """ Optimizer """
        optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate#, weight_decay=0.0005
                                    )

    #     train_set = TensorDataset(train_data, train_dec_label, train_emo_label)
    #     train_loader = Data.DataLoader(train_set, batch_size=32)

    #     test_set = TensorDataset(test_data, test_dec_label, test_emo_label)
    #     test_loader = Data.DataLoader(test_set, batch_size=32)

        for i in range(epoch_size):                                          #多个epoch

            model.train()


            train_loss = 0.0
            train_acc_task = 0.0
            positive_iterations = len(positive_dataloader)
            negative_iterations = len(negative_dataloader)

            # 计算总的迭代次数
            total_iterations = min(positive_iterations, negative_iterations)

            loop = tqdm(enumerate(zip(positive_dataloader, negative_dataloader)), total=total_iterations)


            # 在训练中循环迭代，每次从正样本和负样本数据加载器中分别获取一个批次
            for step,((positive_batch,positive_emo), (negative_batch,negative_emo)) in loop:

                x=torch.cat((positive_batch,negative_batch),dim=0).to(device)
                y1=torch.cat((torch.zeros(len(positive_batch)),torch.ones(len(negative_batch))),dim=0).to(device)
                y2=torch.cat((positive_emo,negative_emo),dim=0).to(device)

                optimizer.zero_grad()

                pred_task, pred_emo, pred_features = model(x)
                loss1 = criterion1(pred_task, y1.long())
                loss2 = criterion2(pred_emo,y2.long())
                loss3 = criterion3(pred_features,y1)

                loss=alpha*loss1+beta*loss2+gamma*loss3

                train_loss += loss.item()

                pred_task = torch.max(pred_task, 1)[1]
                train_correct_task = (pred_task == y1).sum()

                train_acc_task += train_correct_task.item()

                loss.backward()
                optimizer.step()
                loop.set_description(f'Epoch [{i+1} / {epoch_size}]')
                loop.set_postfix({
                        'loss' : '{:.6f}'.format(train_loss/(length1+length2)),
                        'acc_task' : '{:.6f}'.format(train_acc_task*100/(length1+length2))
                                                    })
                if i+1 == epoch_size and save == True:   
                    model_path = './parameter_experiment_new/%s_a=%f_b=%f_g=%f_%d.pkl' % (model_name,alpha,beta,gamma,k+1) #文件夹名称
                    os.makedirs('./parameter_experiment_new', exist_ok=True)   #创建文件夹
                    state = {'model':model.state_dict()
                            }
                    torch.save(state,model_path)


        prob_all = []
        label_all = []
        with torch.no_grad():
            model.eval()  
            for x, y1,y2 in test_loader:
                x, y1, y2 =  Variable(x).to(device), Variable(y1).to(device), Variable(y2).to(device)
                pred_task, pred_emo,pred_features = model(x)
                prob = pred_task.cpu().numpy()
                prob_all.extend(np.argmax(prob,axis=1)) #求每一行的最大值索引
                label_all.extend(y1.cpu().numpy())
            acc_now = accuracy_score(y_true=prob_all, y_pred=label_all)
            f1_now = f1_score(y_true=prob_all, y_pred=label_all, average="macro")
            recall_now = recall_score(y_true=prob_all, y_pred=label_all, average="macro")
            precision_now = precision_score(y_true=prob_all, y_pred=label_all, average="macro")
            print('accuracy:', np.around(acc_now*100, 2))
            print('precision:', np.around(precision_now*100, 2))
            print('recall:', np.around(recall_now*100, 2))
            print('f1:', np.around(f1_now*100, 2))
        acc_all.append(acc_now)
        f1_all.append(f1_now)
        recall_all.append(recall_now)
        precision_all.append(precision_now)




    print('-'*10, model_name, '-'*10)
    print('accuracy:', np.around(np.mean(acc_all)*100, 2), np.std(acc_all))
    print('precision:', np.around(np.mean(precision_all)*100, 2), np.std(precision_all))
    print('recall:', np.around(np.mean(recall_all)*100, 2), np.std(recall_all))
    print('f1:', np.around(np.mean(f1_all)*100, 2), np.std(f1_all))




In [ ]:
MECM_EDNet_learning(1,0.85,0.2)